In [1]:
#import all libraries needed

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV


In [2]:
# Collect data into usable matrices
neg_folder = 'aclImdb/test/neg'
pos_folder = 'aclImdb/test/pos'

# Initialize empty lists to store the text and labels
reviews = []
labels = []

# Read and preprocess positive reviews
for filename in os.listdir(pos_folder):
    with open(os.path.join(pos_folder, filename), 'r', encoding='utf-8') as file:
        review = file.read()
        # Preprocess the review as needed
        # For example, you can remove punctuation and convert to lowercase
        review = review.lower()
        # Append the preprocessed review and label to the lists
        reviews.append(review)
        labels.append(1)  # 1 for positive review


# Read and preprocess negative reviews
for filename in os.listdir(neg_folder):
    with open(os.path.join(neg_folder, filename), 'r', encoding='utf-8') as file:
        review = file.read()
        # Preprocess the review as needed
        # For example, you can remove punctuation and convert to lowercase
        review = review.lower()
        # Append the preprocessed review and label to the lists
        reviews.append(review)
        labels.append(0)  # 0 for negative review

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X = vectorizer.fit_transform(reviews)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)




In [ ]:
# N-Fold Cross validation

In [3]:
# Regression model
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Train the model
logreg.fit(X_train, y_train)

# Score on training and test data
logreg_train_score = logreg.score(X_train, y_train)
logreg_test_score = logreg.score(X_test, y_test)

print(f"Logistic Regression - Training accuracy: {logreg_train_score:.4f}")
print(f"Logistic Regression - Test accuracy: {logreg_test_score:.4f}")


Logistic Regression - Training accuracy: 0.8827
Logistic Regression - Test accuracy: 0.8622


In [4]:
# Create a decision tree classifier
from sklearn.tree import DecisionTreeClassifier

# Create a decision tree classifier
dtree = DecisionTreeClassifier()

# Train the model
dtree.fit(X_train, y_train)

# Score on training and test data
dtree_train_score = dtree.score(X_train, y_train)
dtree_test_score = dtree.score(X_test, y_test)

print(f"Decision Tree - Training accuracy: {dtree_train_score:.4f}")
print(f"Decision Tree - Test accuracy: {dtree_test_score:.4f}")


Decision Tree - Training accuracy: 1.0000
Decision Tree - Test accuracy: 0.6974


In [ ]:
#Support Vector Machines (LinearSVC)
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(imdb_train.data)
X_test = vectorizer.transform(imdb_test.data)
y_train = imdb_train.target
y_test = imdb_test.target

from sklearn.svm import LinearSVC

svc_model = LinearSVC(dual=False)
svc_model.fit(X_train, y_train)

train_accuracy = svc_model.score(X_train, y_train)
test_accuracy = svc_model.score(X_test, y_test)

print(f"LinearSVC - Training accuracy: {train_accuracy:.4f}")
print(f"LinearSVC - Test accuracy: {test_accuracy:.4f}")


In [ ]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# Initialize AdaBoost classifier
ada_clf = AdaBoostClassifier(n_estimators=50, random_state=0)

# Train AdaBoost
ada_clf.fit(X_train, y_train)

# Get training and test accuracies
train_accuracy = ada_clf.score(X_train, y_train)
test_accuracy = ada_clf.score(X_test, y_test)

print("AdaBoost - Training accuracy:", round(train_accuracy, 4))
print("AdaBoost - Test accuracy:", round(test_accuracy, 4))


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)

# Train the classifier
rf_clf.fit(X_train, y_train)

# Get training and test accuracies
train_accuracy_rf = rf_clf.score(X_train, y_train)
test_accuracy_rf = rf_clf.score(X_test, y_test)

print(f"Random Forest - Training accuracy: {train_accuracy_rf:.4f}")
print(f"Random Forest - Test accuracy: {test_accuracy_rf:.4f}")


In [ ]:
#Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting classifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

# Train the Gradient Boosting classifier
gb_clf.fit(X_train, y_train)

# Calculate training accuracy
train_accuracy = gb_clf.score(X_train, y_train)
print(f"Gradient Boosting - Training accuracy: {train_accuracy:.4f}")

# Calculate test accuracy
test_accuracy = gb_clf.score(X_test, y_test)
print(f"Gradient Boosting - Test accuracy: {test_accuracy:.4f}")
